In [1]:
import pandas as pd
import os

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, CategoricalNB

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score

# Load data

In [6]:
CORPUS = []
LABEL = []

In [7]:
# Step 1: get all the fake news text files into a list named corpus, give them label 1 -- fake
dir1 = 'Data/size_normalized_texts/fake/'
list = os.listdir(dir1)
number_files = len(list)

In [8]:
for i in range(number_files):
        title = list[i]
        LABEL.append(1)
        with open(dir1 + title,'r') as reader:

            doc = reader.read()
            doc.lower()
            doc.split()
            reader.close
            CORPUS.append(doc)

In [9]:
# Step 2: repeat the procedure with true news, give them label 0 -- true
dir2 = 'Data/size_normalized_texts/true/'
list = os.listdir(dir2)
number_files = len(list)

for i in range(number_files):
        title = list[i]
        LABEL.append(0)
        with open(dir2 + title,'r') as reader:

            doc = reader.read()
            doc.lower()
            doc.split()
            reader.close
            CORPUS.append(doc)

In [10]:
assert len(CORPUS) == len(LABEL)

# Vectorize words

customize my stop word list based on the context of the NLP application that you are building.

ex.: portuguse_stop_words = ["a", "o","um","uma","pelo", "pela","ou",...]

In [15]:
#stop words in portuguese br:
custom = ['como', 'um', 'que', 'para', 'como', 'uma', 'de', 'ou', 'os', 'em', 'por', 'ma',
                 'se', 'era', 'ela', 'ele', 'era', 'seu', 'sua', 'mai', 'mas', 'nos', 'isso', 'mas', 'entre', 
                 'entre', 'outro', 'na', 'até', 'sobre', 'essa', 'esse', 'quando', 'mais', 'ao', 'dos', 'tem', 
                  'dos', 'está', 'não', 'da', 'disse', 'diz', 'foi', 'caso', 'outra', 'também', 'já', 'ano', 
                'ainda', 'são', 'anos', 'sem', 'porque', 'há', 'eles', 'segundo', 'afirmou', 'acordo', 'muito', 
               'todo', 'muito', 'conta', 'mesmo', 'dele', 'todos', 'dia', 'depois', 'então', 'tudo', 'seja', 'quem'
               'toda', 'após', 'só', 'agora', 'hoje', 'foram', 'grupo', 'assim', 'fazer', 'desde', 'eu', 'estava',
               'estão', 'durante', 'onde', 'das', 'vai', 'onde', 'deu', 'quem', 'qual', 'toda', 'ante', 'outros', 
               'pela', 'fez', 'pelo', 'tinha']

In [11]:
def vectors_and_df(corpus, label):
    """creates vectors for songs and returns dataframe with songs as word vectors 
    by all artists"""
    
    cv = TfidfVectorizer()
    cv.fit(corpus)
    corpus_vecs = cv.transform(corpus)
    
    return pd.DataFrame(corpus_vecs.todense(), index=label, 
                        columns=cv.get_feature_names()), cv

In [12]:
# Store results into dataframe, keep cv for later prediction
df, cv = vectors_and_df(CORPUS, LABEL)

# Split and train data

In [13]:
# Define features and target column
X = df
y = df.index

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    stratify=y)

### Find the best models

In [30]:
pipe = Pipeline([('cvec', CountVectorizer()),    
                 ('lr', LogisticRegression(solver='liblinear'))])

# Tune GridSearchCV
pipe_params = {'cvec__stop_words': [custom],
               'cvec__ngram_range': [(1,1), (2,2), (1,3)],
               'lr__C': [0.01, 1]}

gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train);
print("Best score:", gs.best_score_)
print("Train score", gs.score(X_train, y_train))
print("Test score", gs.score(X_test, y_test))

gs.best_params_

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", 

ValueError: Found input variables with inconsistent numbers of samples: [47035, 5400]

### Find the best parameters

In [16]:
pipe = Pipeline([('cvec', CountVectorizer()),    
                 ('lr', LogisticRegression(solver='liblinear'))])

In [17]:
# Tune GridSearchCV
pipe_params = {'cvec__stop_words': [None, 'english', custom],
               'cvec__ngram_range': [(1,1), (2,2), (1,3)],
               'lr__C': [0.01, 1]}

In [18]:
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(Xtrain, ytrain);
print("Best score:", gs.best_score_)
print("Train score", gs.score(Xtrain, ytrain))
print("Test score", gs.score(Xtest, ytest))

gs.best_params_

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1344, in fit
    accept_large_sparse=solver != 'liblinear')
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py", 

ValueError: Found input variables with inconsistent numbers of samples: [47035, 5760]

In [19]:
MODELS = {
    "MultinomialNB": {"alpha": 0.005},
    "CategoricalNB": {"alpha": 0.01},
    "RandomForestClassifier": {
        "n_estimators": 500,
        "max_depth": 200,
        "max_features": "auto",
        "n_jobs": -1,
        "random_state": 1,
    },
    "LogisticRegression": {"C": 1e6},
}

In [20]:
def train_models(models_params):
    """trains models on corpus and returns dataframe with scores"""
    
    scores = {}
    for model in models_params:
        if model == "LogisticRegression":
            m = LogisticRegression(**models_params[model])
        elif model == "RandomForestClassifier":
            m = RandomForestClassifier(**models_params[model])
        elif model == "MultinomialNB":
            m = MultinomialNB(**models_params[model])
        elif model == "CategoricalNB":
            m = MultinomialNB(**models_params[model])

        m.fit(Xtrain, ytrain)
        score_train = m.score(Xtrain, ytrain)
        score_test = m.score(Xtest, ytest)
        scores[f"{model}"] = {
            "params": models_params[model],
            "train score": score_train,
            "test score": score_test,
            }
    return pd.DataFrame(scores).T

In [21]:
train_models(MODELS)

,params,train score,test score
MultinomialNB,{'alpha': 0.005},0.989757,0.833333
CategoricalNB,{'alpha': 0.01},0.988889,0.839583
RandomForestClassifier,"{'n_estimators': 500, 'max_depth': 200, 'max_f...",1,0.8875
LogisticRegression,{'C': 1000000.0},1,0.913194


In [22]:
# Train on most promising model
model = "LogisticRegression"
m = LogisticRegression(**MODELS[model], max_iter=200)
m.fit(X, y)
m.score(X, y)

1.0

# Confusion Matrix analysis

In [ ]:
def predict(new_text):
    
    """
    Takes the pre-trained model pipeline and predicts new artist based on unseen text.
    
    Parameters
    ----------
    model : Trained scikit-learn model pipeline.
    new_text : str
    
    Returns
    ---------
    prediction : str
    
    """
    article = [new_text]
    # transform song into vector matrix
    new_article_vecs = cv.transform(article)
    ynew = new_article_vecs.todense()
    
    prediction = m.predict(ynew)
    
    return prediction[0]

In [ ]:
if __name__ == '__main__':
    # Whatever happens after this line, execute it when running "python lyrics_classifier.py"
    # and DO NOT execute these lines of code if things from this script are imported from other scripts.
    
    user_input = input('Please Enter Some Text: ')

    prediction = predict(user_input)
    print('Here is your prediction!')
    print(prediction)